In [19]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
import keras

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tf

In [20]:
X_train = load('trainX.npy')
X_test = load('testX.npy')
y_train = load('trainy.npy')
y_test = load('testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(44339, 128, 6) (44339, 20) (4936, 128, 6) (4936, 20)


In [21]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [22]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [23]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=10, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 119, 64)           3904      
                                                                 
 conv1d_9 (Conv1D)           (None, 110, 64)           41024     
                                                                 
 dropout_4 (Dropout)         (None, 110, 64)           0         
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 55, 64)           0         
 1D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 3520)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               450688    
                                                      

In [24]:
history = model.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128, callbacks=callbacks)

Epoch 1/200
278/278 [==============================] - 31s 109ms/step - loss: 1.2469 - accuracy: 0.7038 - val_loss: 0.3802 - val_accuracy: 0.9297
Epoch 2/200
278/278 [==============================] - 32s 116ms/step - loss: 0.2321 - accuracy: 0.9529 - val_loss: 0.1803 - val_accuracy: 0.9621
Epoch 3/200
278/278 [==============================] - 33s 117ms/step - loss: 0.1350 - accuracy: 0.9684 - val_loss: 0.1319 - val_accuracy: 0.9683
Epoch 4/200
278/278 [==============================] - 32s 114ms/step - loss: 0.0990 - accuracy: 0.9765 - val_loss: 0.1016 - val_accuracy: 0.9767
Epoch 5/200
278/278 [==============================] - 35s 125ms/step - loss: 0.0763 - accuracy: 0.9817 - val_loss: 0.0933 - val_accuracy: 0.9777
Epoch 6/200
278/278 [==============================] - 33s 120ms/step - loss: 0.0636 - accuracy: 0.9848 - val_loss: 0.0866 - val_accuracy: 0.9778
Epoch 7/200
278/278 [==============================] - 34s 122ms/step - loss: 0.0529 - accuracy: 0.9876 - val_loss: 0.0720 -

In [25]:
loss, accuracy = model.evaluate(X_validation, y_validation, verbose=True)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=True)
print("Testing Accuracy: {:.4f}".format(accuracy))

278/278 [==============================] - 2s 7ms/step - loss: 0.0492 - accuracy: 0.9869
Validation Accuracy:  0.9869
155/155 [==============================] - 1s 7ms/step - loss: 0.2875 - accuracy: 0.9451
Testing Accuracy: 0.9451
